In [1]:
import os
import csv

#reads all the episodes in a particular folder and reutns them in a array consisting of arrays with tuples
def read_Episodes(baseFolder):
    episodes = []
    for filename in sorted(os.listdir(baseFolder)):
        filePath = os.path.join(baseFolder, filename)
        episode = []
        with open(filePath, "r", newline="") as file:
            reader = csv.reader(file)
            for row in reader:
                observation = [float(epi) for epi in row]
                episode.append(observation)
            episodes.append(episode)        
    return episodes

In [2]:
import d3rlpy
import numpy as np
import os

EPIfolder = "../data/episFormula/3. corrected sign"

epis = read_Episodes(EPIfolder)

observations = []
actions = []
rewards = []
terminals = []

for epi in epis:
    for observation in epi:
        observations.append(observation[:-2])
        actions.append(observation[-2:-1])
        rewards.append(observation[-1])
    terminals += [0] * (len(epi) - 1) + [1]

observations=np.array(observations)
actions=np.array(actions)
rewards=np.array(rewards)
terminals=np.array(terminals)

print(observations.shape)

dataset = d3rlpy.dataset.MDPDataset(
    action_space=d3rlpy.constants.ActionSpace.CONTINUOUS,
    observations=observations,
    actions=actions,
    rewards=rewards,
    terminals=terminals,
)

# use partial episodes as test data
test_episodes = dataset.episodes[:200]

# create action scaler
action_scaler = d3rlpy.preprocessing.MinMaxActionScaler()
observation_scaler = d3rlpy.preprocessing.MinMaxObservationScaler()

# combine FileAdapterFactory and TensorboardAdapterFactory
logger_adapter = d3rlpy.logging.CombineAdapterFactory([
   d3rlpy.logging.FileAdapterFactory(root_dir="logs/d3rlpy_logs"),
   d3rlpy.logging.TensorboardAdapterFactory(root_dir="logs/tensorboard_logs"),
])

# if you don't use GPU, set device=None instead.
#behaviorCloning = d3rlpy.algos.BCConfig(action_scaler=action_scaler, observation_scaler=observation_scaler).create(device='cuda:0')
DDPG = d3rlpy.algos.DDPGConfig(action_scaler=action_scaler, observation_scaler=observation_scaler).create(device='cuda:0')
TD3 = d3rlpy.algos.TD3Config(action_scaler=action_scaler, observation_scaler=observation_scaler).create(device='cuda:0')
SAC = d3rlpy.algos.SACConfig(action_scaler=action_scaler, observation_scaler=observation_scaler).create(device='cuda:0')
BCQ = d3rlpy.algos.BCQConfig(action_scaler=action_scaler, observation_scaler=observation_scaler).create(device='cuda:0')
BEAR = d3rlpy.algos.BEARConfig(action_scaler=action_scaler, observation_scaler=observation_scaler).create(device='cuda:0')
CQL = d3rlpy.algos.CQLConfig(action_scaler=action_scaler, observation_scaler=observation_scaler).create(device='cuda:0')
CRR = d3rlpy.algos.CRRConfig(action_scaler=action_scaler, observation_scaler=observation_scaler).create(device='cuda:0')
CalQL= d3rlpy.algos.CalQLConfig(action_scaler=action_scaler, observation_scaler=observation_scaler).create(device='cuda:0')
AWAC = d3rlpy.algos.AWACConfig(action_scaler=action_scaler, observation_scaler=observation_scaler).create(device='cuda:0')
PLAS = d3rlpy.algos.PLASConfig(action_scaler=action_scaler, observation_scaler=observation_scaler).create(device='cuda:0')
PLASP = d3rlpy.algos.PLASWithPerturbationConfig(action_scaler=action_scaler, observation_scaler=observation_scaler).create(device='cuda:0')
TD3BC = d3rlpy.algos.TD3PlusBCConfig(action_scaler=action_scaler, observation_scaler=observation_scaler).create(device='cuda:0')
PRDC = d3rlpy.algos.PRDCConfig(action_scaler=action_scaler, observation_scaler=observation_scaler).create(device='cuda:0')
ReBRAC = d3rlpy.algos.ReBRACConfig(action_scaler=action_scaler, observation_scaler=observation_scaler).create(device='cuda:0')
IQL = d3rlpy.algos.IQLConfig(action_scaler=action_scaler, observation_scaler=observation_scaler).create(device='cuda:0')

Models = [DDPG, TD3, SAC, BCQ, BEAR, CQL, CRR, CalQL, AWAC, PLAS, PLASP, TD3BC, PRDC, ReBRAC, IQL]

for rlModel in Models:
    print("Training model: ", rlModel.__class__.__name__)
    result = rlModel.fit(
        dataset,
        n_steps=200000,
        n_steps_per_epoch=1000,
        evaluators={
           'td_error': d3rlpy.metrics.TDErrorEvaluator(test_episodes),
           'value_scale': d3rlpy.metrics.AverageValueEstimationEvaluator(test_episodes),
           'discounted_advantage': d3rlpy.metrics.DiscountedSumOfAdvantageEvaluator(test_episodes),
           'initial_state': d3rlpy.metrics.InitialStateValueEstimationEvaluator(test_episodes),
           'diff_eval': d3rlpy.metrics.ContinuousActionDiffEvaluator(test_episodes)
        },
        logger_adapter=logger_adapter,
    )

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
c:\Users\NilsWindows\Desktop\research2.0\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(23981, 7)
2025-12-05 23:19.03 [info     ] Signatures have been automatically determined. action_signature=Signature(dtype=[dtype('float64')], shape=[(1,)]) observation_signature=Signature(dtype=[dtype('float64')], shape=[(7,)]) reward_signature=Signature(dtype=[dtype('float64')], shape=[(1,)])
2025-12-05 23:19.03 [info     ] Action size has been automatically determined. action_size=1
Training model:  DDPG
2025-12-05 23:19.03 [info     ] dataset info                   dataset_info=DatasetInfo(observation_signature=Signature(dtype=[dtype('float64')], shape=[(7,)]), action_signature=Signature(dtype=[dtype('float64')], shape=[(1,)]), reward_signature=Signature(dtype=[dtype('float64')], shape=[(1,)]), action_space=<ActionSpace.CONTINUOUS: 1>, action_size=1)
2025-12-05 23:19.03 [debug    ] Fitting observation scaler...  observation_scaler=min_max
2025-12-05 23:19.03 [debug    ] Fitting action scaler...       action_scaler=min_max
2025-12-05 23:19.03 [debug    ] Building models...          

Epoch 1/200: 100%|██████████| 1000/1000 [00:17<00:00, 57.02it/s, critic_loss=0.236, actor_loss=-2.01]


2025-12-05 23:19.27 [info     ] DDPG_20251205231906: epoch=1 step=1000 epoch=1 metrics={'time_sample_batch': 0.004881596088409424, 'time_algorithm_update': 0.012094255924224853, 'critic_loss': 0.23970390455424787, 'actor_loss': -2.027244101472199, 'time_step': 0.01722714161872864, 'td_error': 1.6770780791805877, 'value_scale': 4.006850385546609, 'discounted_advantage': -7.005441939245865, 'initial_state': 3.9310083389282227, 'diff_eval': 113474.19222585033} step=1000
2025-12-05 23:19.27 [info     ] Model parameters are saved to logs/d3rlpy_logs\DDPG_20251205231906\model_1000.d3


Epoch 2/200: 100%|██████████| 1000/1000 [00:16<00:00, 59.78it/s, critic_loss=0.949, actor_loss=-6.37]


...

2025-12-07 01:18.02 [info     ] IQL_20251206235012: epoch=198 step=198000 epoch=198 metrics={'time_sample_batch': 0.0047635862827301026, 'time_algorithm_update': 0.017011022806167602, 'critic_loss': 3.8095924127697947, 'q_loss': 3.59548973608017, 'v_loss': 0.21410268051177264, 'actor_loss': -2.9287909581959246, 'time_step': 0.02202396869659424, 'td_error': 2.2348912399623493, 'value_scale': 4.600328320220475, 'discounted_advantage': -2.418209725765147, 'initial_state': 6.575725078582764, 'diff_eval': 383.4359061903748} step=198000
2025-12-07 01:18.02 [info     ] Model parameters are saved to logs/d3rlpy_logs\IQL_20251206235012\model_198000.d3


Epoch 199/200: 100%|██████████| 1000/1000 [00:23<00:00, 42.48it/s, critic_loss=3.82, q_loss=3.6, v_loss=0.215, actor_loss=-2.73]


2025-12-07 01:18.30 [info     ] IQL_20251206235012: epoch=199 step=199000 epoch=199 metrics={'time_sample_batch': 0.004743489980697632, 'time_algorithm_update': 0.018152740955352784, 'critic_loss': 3.8161244480609895, 'q_loss': 3.6008103934526443, 'v_loss': 0.21531405185163022, 'actor_loss': -2.7327598309516907, 'time_step': 0.02316284990310669, 'td_error': 2.128642262860288, 'value_scale': 4.3788001049697955, 'discounted_advantage': -1.897362978628612, 'initial_state': 6.328591823577881, 'diff_eval': 369.6914758619569} step=199000
2025-12-07 01:18.30 [info     ] Model parameters are saved to logs/d3rlpy_logs\IQL_20251206235012\model_199000.d3


Epoch 200/200: 100%|██████████| 1000/1000 [00:24<00:00, 40.26it/s, critic_loss=3.9, q_loss=3.68, v_loss=0.22, actor_loss=-2.8]  


2025-12-07 01:18.58 [info     ] IQL_20251206235012: epoch=200 step=200000 epoch=200 metrics={'time_sample_batch': 0.006180122852325439, 'time_algorithm_update': 0.017983187913894652, 'critic_loss': 3.9050112788677214, 'q_loss': 3.6844584316015245, 'v_loss': 0.22055284932255745, 'actor_loss': -2.8101650324761867, 'time_step': 0.024439037322998045, 'td_error': 2.0670665125998755, 'value_scale': 4.37828069248499, 'discounted_advantage': -1.7019425175642138, 'initial_state': 6.083855152130127, 'diff_eval': 352.70337413104585} step=200000
2025-12-07 01:18.58 [info     ] Model parameters are saved to logs/d3rlpy_logs\IQL_20251206235012\model_200000.d3
